In [163]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import pandas as pd

In [164]:
train  = np.genfromtxt("train.csv",delimiter=",")

In [165]:
test = np.genfromtxt("test.csv",delimiter=",")

In [166]:
X_train = train[:,0:784]/255
Y_train = train[:,784].astype(int)
X_test = test[:,0:784]/255
Y_test = test[:,784].astype(int)

In [167]:
X_train.shape

(20000, 784)

In [168]:
X_test.shape

(10000, 784)

In [169]:
len(X_train)

20000

In [223]:
class SVM_classifier:
    def __init__(self,class_indices = (0,2),C=1, nf=784, bs=100, lamb=1, iters=500):
        self.weights = np.zeros(nf)
        self.C = C
        self.bias = 0
        self.bs = bs
        self.lamb = lamb
        self.iters = iters
        self.class_indices = class_indices

    def fit(self, X_train, Y_train):
        for i in range(1, self.iters + 1):
            eta = 1 / (self.lamb * i)
            idx = np.random.choice(
                np.arange(len(X_train)), self.bs, replace=False)
            X_sample = X_train[idx]
            Y_sample = Y_train[idx]
            T = Y_sample * (np.matmul(X_sample, self.weights) + self.bias)
            is_bad = np.greater_equal(1, T)
            self.bias = self.bias - eta * (np.dot(is_bad, -Y_sample))
            
            self.weights = self.weights - eta * (
                self.lamb * self.weights -
                (np.dot(is_bad, X_sample * Y_sample[:, None])*self.C)
            )

    def predict(self, X):
        p = np.matmul(X, self.weights) + self.bias
        p[p < 0] = -1
        p[p >= 0] = 1
        return p.astype(int)

In [228]:
class Multi_svm:
    def __init__(self,classes = range(0,10)):
        # conv classif(a,b) exists if a<b ; out 1 if a and -1 if b
        self.classifiers = []
        self.classes = classes
        self.class_indices = range(len(self.classes))
        for cls1 in self.class_indices:
            for cls2 in self.class_indices:
                if cls1 < cls2:
                    new_clf = SVM_classifier(class_indices=(cls1,cls2))
                    self.classifiers.append(new_clf)
    def fit(self,X_train,Y_train):
        for cur_clf in self.classifiers:
            cls1,cls2 = cur_clf.class_indices
            X_sub = X_train[(Y_train==self.classes[cls1]  )| (Y_train==self.classes[cls2]) ] 
            Y_sub = np.copy(Y_train[(Y_train==self.classes[cls1]  )| (Y_train==self.classes[cls2]) ] )
            Y_sub[Y_sub==self.classes[cls1]] = 1
            Y_sub[Y_sub==self.classes[cls2]] = -1
            cur_clf.fit(X_sub,Y_sub)
    def predict(self,X):
        num_samples = X.shape[0]
        pred_matrix = np.zeros(shape=(num_samples,len(self.classes)),dtype=int)
        for cur_clf in self.classifiers:
            pr = cur_clf.predict(X)
            pr[pr==1] = cur_clf.class_indices[0]
            pr[pr==-1] = cur_clf.class_indices[1] #order of these two matters xD
            pred_matrix[np.arange(0,num_samples),pr] += 1
        maxy = np.argmax(pred_matrix,axis=1)
        indice_to_name = [self.classes[i] for i in maxy]
        return np.asarray(maxy)
    

In [229]:
mm = Multi_svm()

In [230]:
mm.fit(X_train,Y_train)

In [231]:
import sklearn

pred = mm.predict(X_test)

In [232]:
print(sklearn.metrics.accuracy_score(Y_test,pred))

0.8943


In [233]:
print(sklearn.metrics.classification_report(Y_test,pred))

             precision    recall  f1-score   support

          0       0.95      0.96      0.95       980
          1       0.90      0.98      0.94      1135
          2       0.93      0.84      0.88      1032
          3       0.89      0.88      0.88      1010
          4       0.89      0.90      0.89       982
          5       0.81      0.88      0.84       892
          6       0.92      0.93      0.92       958
          7       0.93      0.88      0.90      1028
          8       0.88      0.82      0.85       974
          9       0.85      0.88      0.87      1009

avg / total       0.90      0.89      0.89     10000

